# Fiji Cellprofiler Python Connection
## Import packages:

In [2]:
import os
import imagej
from pathlib import Path
import sys
import csv
import glob

## Function to select a file via a dialogue window and return the file name:

In [3]:
%gui qt 

#how to define gui 

from PyQt5.QtWidgets import QFileDialog #import QFileDilogue function 

def get_filename(dir=None): #Create a function to select a file via a dialogue window and return file name
    if dir is None: dir ='./'
    fname = QFileDialog.getOpenFileName(None, "Select data file...", 
                dir, filter="All files (*);; SM Files (*.sm)")
    return fname[0]

## Function to select a folder via a dialogue window and return the folder path:

In [4]:
%gui qt 

#how to define gui 

from PyQt5.QtWidgets import QFileDialog #import QFileDilogue function

def get_folder(dir=None): #function to select a folder via a dialogue and return folder path
    if dir is None: dir ='./'
    fname = QFileDialog.getExistingDirectory(None, "Select Directory...")
    return fname

## Function to modify the macro file with chosen directories:

In [5]:
def modify_macro(macro, directory_in, directory_out): #function to mdify the macro with updated information of directries output/input
    
    #To separate the method by OS
    #if os.name == 'nt':         
    
    path_is_in=os.path.realpath(directory_in) #input directory
    path_is_in=path_is_in.replace('\\', '/') #Windows paths to work in macro. In windows paths in the shell command contain two backslashes
    #print(path_is_in)
    path_is_out=os.path.realpath(directory_out) #output directory
    path_is_out=path_is_out.replace('\\', '/')
    #print(path_is_out)
    if not os.path.exists(macro): #if variable doesnt have macro file saved, print 'macro file not there' (to make sure its selected)
        print('Macro file not there...')
        return False
    else:
        with open(macro, 'r+') as fp:  #open it in a read and write format
            lines=fp.readlines()   #reads it line by line splits macro in 'inputDirectory' or 'outputDirectory' to insert a new one
            for i, line in enumerate(lines):   
                if line.startswith('inputDirectory'):
                    lines[i]='inputDirectory = "' + path_is_in + '/";\n'
                elif line.startswith('outputDirectory'):
                    lines[i]='outputDirectory = "' + path_is_out + '/";\n'
        with open(macro, 'r+') as fp:
            fp.writelines(lines)  #we are adding modified line variable back to the file
        return macro

## Create a function that generates a csv file with paths to images (to be analyzed with Cellprofiler):

In [6]:
def make_csv(var1):   
    path_image=os.path.join(var1 + '/image_list.csv')
    print('Image list csv file:\n' + path_image)
    if not os.path.exists(path_image):
        with open(path_image, 'w') as fp:
            pass
    else:
        os.remove(path_image)
        with open(path_image,'w') as fp:
            pass
    path_image_files=os.path.join(var1, '*.tif')
    for file in glob.glob(path_image_files):
        #print('Writing...\n' + file)
        path_is=os.path.realpath(file)
        #print('Real path:\n' + path_is)
        with open(path_image, 'a') as image_list:
            dummywriter=csv.writer(image_list, delimiter=' ',
                                    quotechar='|', quoting=csv.QUOTE_MINIMAL)
            dummywriter.writerow([path_is])
    return path_image

## Initialize ImageJ using path to your ImageJ application:

In [7]:
ij = imagej.init('Downloads/Fiji.app')
ij.getVersion() #Shows version of initialized ImageJ

'2.0.0-rc-69/1.52p'

## Choose, open, and read a macro file:

In [9]:
macro_path=get_filename() #get_filename() is the function we have created above
macro=open(macro_path) #opens the encoded file with the macro path
macro=macro.read() #Decodes macro file to make it readable
print('Macro selected:\n' + macro_path) #prints macro path

Macro selected:
/Users/sbp/notebook/fiji_cp_integration/umed3.ijm


## Choose a folder with images to be analyzed with FIJI:

In [12]:
fiji_in=get_folder() #Choose a folder with the get_folder() function created above 
print('Image is in:\n' + fiji_in) #Print folder path

Image is in:
/Users/sbp/Documents/Images 042 TEST/Input 042 TEST


## Create a folder where the images will be stored after they are analyzed with FIJI:

In [13]:
fiji_out = str(fiji_in + '_post_fiji') #selecting a folder with a new name where imajes will be saved after FIJI analysis
if not os.path.exists(fiji_out):  #the first time the code is run this folder doesn't exist so it will be created
    os.makedirs(fiji_out) 
print('Image will be saved in:\n' + fiji_out) #print folder path

Image will be saved in:
/Users/sbp/Documents/Images 042 TEST/Input 042 TEST_post_fiji


## Modify the macro file with new directories:

In [14]:
try:
    macro_path=modify_macro(macro_path, fiji_in, fiji_out) #calling the modify_macro() function created above with the path to macro, input, and output folders
    macro=open(macro_path)
    macro=macro.read()
except:                                
    print("Unexpected error:", sys.exc_info()[0]) #in case there is an error print "unexpected error"
    raise

## Run the macro:

In [16]:
ij.py.run_macro(macro)

<org.scijava.script.ScriptModule at 0x11ed486b0 jclass=org/scijava/script/ScriptModule jself=<LocalRef obj=0x7fc28e47df98 at 0x11ba43790>>

## Create an output folder to save results after Cellprofiler:

In [17]:
cp_out = str(fiji_in + '_post_Cellprofiler') 
if not os.path.exists(cp_out):
    os.makedirs(cp_out)
print('The final results will be saved in:\n' + cp_out)

The final results will be saved in:
/Users/sbp/Documents/Images 042 TEST/Input 042 TEST_post_Cellprofiler


## Create a csv file with image paths (for Cellprofiler):

In [18]:
path_image = make_csv(fiji_out) #calling the make_csv() function created above

Image list csv file:
/Users/sbp/Documents/Images 042 TEST/Input 042 TEST_post_fiji/image_list.csv


## Choose a pipeline file (for Cellprofiler):

In [19]:
pipe = get_filename() #calling get_filename() function created above
print('Pipeline:\n' + pipe) #printing pipeline path

Pipeline:
/Users/sbp/Downloads/speckle_v2.cppipe


## Modifying the paths for the shell command (needed for windows only):

In [58]:
#pipe = pipe.replace(r'/','/')   #Four backslashes are read as two backslashes in the shell command 
#cp_in = path_image.replace(r'/','/')
#cp_out = cp_out.replace(r'/','/')
#print(pipe) #printing out paths to see what they look like
#print(cp_in)
#print(cp_out)

\\Users\\sbp\\Downloads\\speckle_v2.cppipe
/Users/sbp/Documents/Images 042 TEST/Input 042 TEST_post_fiji/image_list.csv
\\Users\\sbp\\Documents\\Images 042 TEST\\Input 042 TEST_post_Cellprofiler


## Look for the cellprofiler.exe in any given device and modify the pathname to be readable in the shell:

In [51]:
tmp=str(glob.glob('/*/*/CellProfiler'))#searching for cellprofiler within the cdrive
print(tmp)
path_tmp=str(os.path.normpath(tmp).strip("['']"))  #when you find the file it comes out as a list with one element. Eliminating this list
path_list=path_tmp.split('\\') #splitting the part that contains program files  
dir_name=path_list[1]
if 'x86' in dir_name:
    path_list[1:2] = ['progra~2']   #defined differently in the cdrive than in the shell
    s='\\\\'
    cp=s.join(path_list)
else:
    path_list[1:2] = ['progra~1'] #defined differently in the cdrive than in the shell
    s='\\\\'
    cp=s.join(path_list)
print(cp)

[]


IndexError: list index out of range

In [50]:
tmp

'[]'

In [47]:
path_tmp

''

In [48]:
path_list

['']

In [34]:
#tmp=str(glob.glob('/*/*/CellProfiler.exe'))#searching for cellprofiler within the cdrive
tmp='Users/sbp/Downloads/CellProfiler-3.1.9.app'
#path_tmp=str(os.path.normpath(tmp).strip("['']"))  #when you find the file it comes out as a list with one element. Eliminating this list
path_list=path_tmp.split('/') #splitting the part that contains program files  
dir_name=path_list[1]
if 'x86' in dir_name:
    path_list[1:2] = ['progra~2']   #defined differently in the cdrive than in the shell
    s='\\\\'
    cp=s.join(path_list)
else:
    path_list[1:2] = ['progra~1'] #defined differently in the cdrive than in the shell
    s='\\\\'
    cp=s.join(path_list)
print(cp)

Downloads\\progra~1


In [39]:
print(path_list)
print(path_tmp)
print(tmp) #I need to use open 

['Downloads', 'progra~1']
Downloads/CellProfiler-3.1.9.app
Downloads/CellProfiler-3.1.9.app


## Running the headless mode of Cellprofiler in the shell (so that t can run in the same notebook as FIJI):

In [55]:
cmd= 'open -a /Users/sbp/Downloads/CellProfiler-3.1.9.app -r -c -p pipe --file-list=cp_in -o cp_out'

In [56]:
#cmd = str(cp + ' -c -r -p ' + pipe + ' --file-list=' + cp_in + ' -o ' + cp_out)
print(cmd)
os.system(cmd) #this is how you run it in the shell
#os.system('cellprofiler -r -c -p ~/project/speckle_v3.cppipe --file-list=/home/h3/project/image_out_fiji/image_list.csv -o ~/project/image_out_cp')

open -a /Users/sbp/Downloads/CellProfiler-3.1.9.app -r -c -p pipe --file-list=cp_in -o cp_out


256